<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C3/W3/ungraded_labs/C3_W3_Lab_5_sarcasm_with_bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/doantronghieu/DEEP-LEARNING/main/helper_DL.py
!pip install colorama
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':15})
import seaborn           as sns
sns.set()
import helper_DL as helper

# Ungraded Lab: Training a Sarcasm Detection Model using Bidirectional LSTMs

In this lab, you will revisit the [News Headlines Dataset for Sarcasm Detection](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection/home) dataset and use it to train a Bi-LSTM Model.


## Download the Dataset

First, you will download the JSON file and extract the contents into lists.

In [ ]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

In [ ]:
import json

# Load the JSON file
with open('./sarcasm.json', 'r') as f:
    datastore = json.load(f)

# Initialize the lists
sentences = []
labels    = []

# Collect sentences and labels into the lists
for item in datastore:
    sentences.append(item['headline'])
    labels   .append(item['is_sarcastic'])

## Split the Dataset

You will then split the lists into train and test sets.

In [ ]:
training_size = 20000

# Split the sentences
training_sentences = sentences[0:training_size]
testing_sentences  = sentences[training_size:]

# Split the labels
training_labels = labels[0:training_size]
testing_labels  = labels[training_size:]

## Data preprocessing

Next, you will generate the vocabulary and padded sequences.

In [ ]:
import numpy as np
import tensorflow.keras.preprocessing as tfkp

vocab_size    = 10000 # Vocabulary size of tokenizer
max_length    = 120   # Maximum length of the padded sequences
trunc_type   = 'post'
padding_type = 'post'
oov_tok      = '<OOV>'

# Initialize the Tokenizer class
tokenizer = tfkp.text.Tokenizer(num_words = vocab_size, oov_token = oov_tok)

# Generate the word index dictionary
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded    = tfkp.sequence.pad_sequences(training_sequences,
                                                 maxlen = max_length,
                                                 padding = padding_type,
                                                 truncating = trunc_type) 

# Generate and pad the testing sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded    = tfkp.sequence.pad_sequences(testing_sequences,
                                                maxlen = max_length,
                                                padding = padding_type,
                                                truncating = trunc_type) 

# Convert the labels lists into numpy arrays
training_labels = np.array(training_labels)
testing_labels  = np.array(testing_labels)

## Build and Compile the Model

The architecture here is almost identical to the one you used in the previous lab with the IMDB Reviews. Try to tweak the parameters and see how it affects the training time and accuracy (both training and validation).

In [ ]:
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow import nn
from tensorflow.keras import layers, losses, optimizers, models, Model

# Hyper parameters
EMBEDDING_DIM = 16
LSTM_DIM      = 32
DENSE_DIM     = 24

# Buid the model
model_lstm = models.Sequential([
    layers.Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length),
    layers.Bidirectional(layers.LSTM(LSTM_DIM)),  
    layers.Dense(DENSE_DIM, activation = nn.relu),
    layers.Dense(1, activation = nn.sigmoid)                         
])

model_lstm.summary()

# Set the training parameters
model_lstm.compile(loss = losses.binary_crossentropy,
                   optimizer = optimizers.Adam(),
                   metrics = ['accuracy'])

## Train the Model

In [ ]:
NUM_EPOCHS = 10

# Train the model
history_lstm = model_lstm.fit(training_padded, 
                              training_labels, 
                              epochs=NUM_EPOCHS, 
                              validation_data=(testing_padded, testing_labels))

helper.plot_history_curves(history_lstm)